In [680]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn import linear_model
from sklearn.utils.deprecation import *


In [681]:
train_dataset = pd.read_csv('Test Titanic Datset/train.csv')
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [682]:
train_dataset.shape

(891, 12)

In [683]:
X_train = train_dataset.drop(['Survived'], axis=1)
y_train = train_dataset['Survived']
print(X_train.shape), print(y_train.shape)

(891, 11)
(891,)


(None, None)

In [684]:
X_test = pd.read_csv('Test Titanic Datset/test.csv')
y_test = pd.read_csv('Test Titanic Datset/gender_submission.csv')

In [685]:
print(X_test.shape), print(y_test.shape)

(418, 11)
(418, 2)


(None, None)

In [686]:
###Concatenamos los dos datasets para hacer las transformaciones a la vez en ambos

In [687]:
raw_data = pd.concat([X_train, X_test], axis=0, ignore_index = True)
raw_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [688]:
### Eliminamos las columnas que no necesitamos para el K-Fold CV

In [689]:
raw_data = raw_data.drop(['PassengerId', 'Name','Ticket','Cabin'], axis=1)
raw_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [690]:
### Transformamos los campos de texto a numericos para el algoritmo

In [691]:
raw_data['Sex'] = raw_data['Sex'].map({'male' : 0, 'female' : 1}).astype('float64')
raw_data['Embarked'] = raw_data['Embarked'].map({'C' : 0, 'S' : 1, 'Q' : 2}).astype('float64')
raw_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0.0,22.0,1,0,7.2500,1.0
1,1,1.0,38.0,1,0,71.2833,0.0
2,3,1.0,26.0,0,0,7.9250,1.0
3,1,1.0,35.0,1,0,53.1000,1.0
4,3,0.0,35.0,0,0,8.0500,1.0


In [692]:
"""Los campos vacios los completamos:
- Age y Fare los completamos con las medianas.
- Embarked lo cumplimentamos a C
"""


'Los campos vacios los completamos:\n- Age y Fare los completamos con las medianas.\n- Embarked lo cumplimentamos a C\n'

In [693]:
raw_data.isnull().sum()

Pclass        0
Sex           0
Age         263
SibSp         0
Parch         0
Fare          1
Embarked      2
dtype: int64

In [694]:
raw_data['Age'] = raw_data['Age'].fillna(raw_data.Age.median())
raw_data['Fare'] = raw_data['Fare'].fillna(raw_data.Fare.median())
raw_data['Embarked'] = raw_data['Embarked'].fillna(0)

In [695]:
"""Volvemos a separar los datasets de train y test una vez transformados"""

'Volvemos a separar los datasets de train y test una vez transformados'

In [696]:
X_train = raw_data[:891]
X_test = raw_data[891:]

In [697]:
"""Formateo bien las etiquetas de test"""

'Formateo bien las etiquetas de test'

In [698]:
y_test = y_test.drop(['PassengerId'], axis=1)

In [699]:
"""Hago el K-Fold CV para ver los ratios en cada uno de los subconjuntos. Escogemos 5 subconjuntos. El clasificador es lineal con SGD"""

'Hago el K-Fold CV para ver los ratios en cada uno de los subconjuntos. Escogemos 5 subconjuntos. El clasificador es lineal con SGD'

In [700]:
from sklearn.model_selection import cross_val_score
clf = linear_model.SGDClassifier(alpha=0.001, random_state=42)
clf.fit(X_train, y_train)
print("%0.2f de precisión en train set" % clf.score(X_train,y_train))
print("%0.2f de precisión en test set" % clf.score(X_test,y_test))
print("Scores ACC en la CV: " + str(cross_val_score(clf, X_train, y_train, cv=5)))


0.77 de precisión en train set
0.80 de precisión en test set
Scores ACC en la CV: [0.58100559 0.69662921 0.40449438 0.76966292 0.71910112]


In [701]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.72 accuracy with a standard deviation of 0.03


In [702]:
from sklearn import metrics
print("Scores F1 en la CV: " + str(cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')))

Scores F1 en la CV: [0.55898295 0.61983863 0.31607945 0.76641168 0.67110126]


In [703]:
"""Hacemos una prueba con los datasets correctamente escalados para mejorar la precisión"""

'Hacemos una prueba con los datasets correctamente escalados para mejorar la precisión'

In [704]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = linear_model.SGDClassifier(alpha=0.001, random_state=42).fit(X_train_transformed, y_train)
print("%0.2f de precisión en train escalado" % clf.score(X_train_transformed, y_train))
X_test_transformed = scaler.transform(X_test)
print("%0.2f de precisión en test escalado" % clf.score(X_test_transformed, y_test))

0.79 de precisión en train escalado
0.99 de precisión en test escalado


In [705]:
print("Scores ACC en la CV: " + str(cross_val_score(clf, X_train_transformed, y_train, cv=5)))

Scores ACC en la CV: [0.79329609 0.80337079 0.76404494 0.75842697 0.81460674]
